In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset  = keras.datasets.fashion_mnist

In [3]:
(train_images, train_labels),(test_images,test_lables) = dataset.load_data()

#### Scaling Down Images

In [4]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [5]:
## Getting the pixels of images
train_images[0].shape

(28, 28)

In [6]:
## Resizing the images
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [7]:
def built_model(hyperparams):
    model = keras.Sequential([
        
        keras.layers.Conv2D(
        filters = hyperparams.Int('conv_1_filter',min_value=32,max_value=128,step=16),
        kernel_size=hyperparams.Choice('conv_1_kernel',values=[3,5]),## Size of Filter
        activation='relu',
        input_shape=(28,28,1)
        ),
        
        keras.layers.Conv2D(
        filters = hyperparams.Int('conv_2_filter',min_value=32,max_value=64,step=16),
        kernel_size=hyperparams.Choice('conv_2_kernel',values=[3,5]),## Size of Filter
        activation='relu',
        ),
        
        keras.layers.Flatten(),
        keras.layers.Dense(
        units=hyperparams.Int('dense_1_units',min_value=32,max_value=128,step=16),
        activation='relu'
        ),
        
        keras.layers.Dense(10,activation='softmax')
    ])
    
    model.compile(optimizer = keras.optimizers.Adam(hyperparams.Choice('learning_rate',values=[1e-2,1e-3])),
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])
    return model
    

In [8]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [9]:
tuner_search = RandomSearch(built_model,objective='val_accuracy',max_trials=5,directory='output',project_name='MNIST_FASHION')

INFO:tensorflow:Reloading Oracle from existing project output\MNIST_FASHION\oracle.json
INFO:tensorflow:Reloading Tuner from output\MNIST_FASHION\tuner0.json


In [10]:
print(tf.config.list_physical_devices('GPU'))
print(tf.test.is_built_with_cuda())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True


In [11]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1) 
## It will find the best parameter. 
## NOTE: BY DEFAULT IT WILL RUN ONLY FOR 2 EPOCHS, SO CHANGE ACCORDINGLY TO IT.

INFO:tensorflow:Oracle triggered exit


In [12]:
model = tuner_search.get_best_models(num_models=1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        55344     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               3096688   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,154,442
Trainable params: 3,154,442
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
